## 1. **CÁCH NHẬN BIẾT OUTLIER:**



Có 3 phương pháp chính để phát hiện outlier:

a) **Phương pháp IQR (Interquartile Range)**:
```python
def detect_outliers_iqr(data, column):
    # Tính Q1, Q3 và IQR
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    
    # Tính giới hạn trên và dưới
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Xác định outliers
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    
    print(f'Số lượng outliers: {len(outliers)}')
    print(f'Giới hạn dưới: {lower_bound}')
    print(f'Giới hạn trên: {upper_bound}')
    
    return outliers, lower_bound, upper_bound

# Sử dụng
outliers, lower, upper = detect_outliers_iqr(data, 'length')
```

b) **Phương pháp Z-score**:
```python
def detect_outliers_zscore(data, column, threshold=3):
    # Tính Z-score
    z_scores = (data[column] - data[column].mean()) / data[column].std()
    
    # Xác định outliers (|z| > 3)
    outliers = data[abs(z_scores) > threshold]
    
    print(f'Số lượng outliers: {len(outliers)}')
    return outliers

# Sử dụng
outliers_z = detect_outliers_zscore(data, 'length')
```

c) **Trực quan hóa** để phát hiện:
```python
def visualize_outliers(data, column):
    plt.figure(figsize=(15, 5))
    
    # Box Plot
    plt.subplot(131)
    sns.boxplot(y=data[column])
    plt.title('Box Plot')
    
    # Histogram
    plt.subplot(132)
    sns.histplot(data[column], kde=True)
    plt.title('Histogram')
    
    # Q-Q Plot
    plt.subplot(133)
    stats.probplot(data[column], dist="norm", plot=plt)
    plt.title('Q-Q Plot')
    
    plt.tight_layout()
    plt.show()

# Sử dụng
visualize_outliers(data, 'length')
```

2. **CÁCH XỬ LÝ OUTLIER:**

a) **Loại bỏ (Remove)**:
```python
def remove_outliers(data, column):
    # Phát hiện outliers
    outliers, lower, upper = detect_outliers_iqr(data, column)
    
    # Loại bỏ outliers
    data_clean = data[(data[column] >= lower) & (data[column] <= upper)]
    
    print(f'Số mẫu ban đầu: {len(data)}')
    print(f'Số mẫu sau khi xử lý: {len(data_clean)}')
    
    return data_clean

# Sử dụng
data_cleaned = remove_outliers(data, 'length')
```

b) **Thay thế (Replace)**:
```python
def replace_outliers(data, column):
    # Phát hiện outliers
    outliers, lower, upper = detect_outliers_iqr(data, column)
    
    # Copy data
    data_clean = data.copy()
    
    # Thay thế outliers bằng giới hạn
    data_clean.loc[data_clean[column] > upper, column] = upper
    data_clean.loc[data_clean[column] < lower, column] = lower
    
    return data_clean

# Sử dụng
data_cleaned = replace_outliers(data, 'length')
```

c) **Biến đổi dữ liệu**:
```python
def transform_data(data, column):
    # Log transform
    data_log = np.log1p(data[column])
    
    # Box-Cox transform
    data_boxcox, lambda_param = stats.boxcox(data[column])
    
    return data_log, data_boxcox

# Sử dụng
data_log, data_boxcox = transform_data(data, 'length')
```

3. **SO SÁNH TRƯỚC VÀ SAU XỬ LÝ:**

```python
def compare_before_after(data, column):
    # Xử lý outliers
    data_cleaned = remove_outliers(data, column)
    
    # So sánh thống kê
    print("\nThống kê trước xử lý:")
    print(data[column].describe())
    print("\nThống kê sau xử lý:")
    print(data_cleaned[column].describe())
    
    # Vẽ đồ thị so sánh
    plt.figure(figsize=(15, 5))
    
    # Box Plot
    plt.subplot(131)
    plt.boxplot([data[column], data_cleaned[column]], 
                labels=['Trước', 'Sau'])
    plt.title('So sánh Box Plot')
    
    # Histogram
    plt.subplot(132)
    plt.hist(data[column], bins=50, alpha=0.5, label='Trước')
    plt.hist(data_cleaned[column], bins=50, alpha=0.5, label='Sau')
    plt.legend()
    plt.title('So sánh Histogram')
    
    # KDE Plot
    plt.subplot(133)
    sns.kdeplot(data[column], label='Trước')
    sns.kdeplot(data_cleaned[column], label='Sau')
    plt.title('So sánh KDE')
    
    plt.tight_layout()
    plt.show()

# Sử dụng
compare_before_after(data, 'length')
```

Các chỉ số quan trọng cần so sánh:
1. Số lượng mẫu trước và sau
2. Các thống kê cơ bản (mean, median, std)
3. Phân phối dữ liệu (từ đồ thị)
4. Tác động đến biến mục tiêu (nếu có)

Chọn phương pháp xử lý phù hợp dựa trên:
- Bản chất dữ liệu
- Số lượng outliers
- Mục đích phân tích
- Ảnh hưởng đến mô hình

# 2. Chi tiết 

## 2.1 Phát hiện **outliers** bằng phương pháp **IQR** (cực dễ hiểu cho học sinh cấp 2)



---

**Outliers** (giá trị ngoại lai) là những giá trị "lạc loài" trong một dãy số. Chúng thường quá cao hoặc quá thấp so với hầu hết các giá trị khác. Ví dụ:

- Trong lớp học, nếu chiều cao trung bình của các bạn là 150-160 cm, nhưng có một bạn cao đến 190 cm hoặc thấp hơn 120 cm, thì bạn đó có thể là **outlier**.

Vậy làm sao để tìm ra những outliers? Hôm nay mình sẽ giới thiệu một cách cực kỳ hiệu quả và dễ hiểu, đó là phương pháp **IQR** (Interquartile Range).

---

### Bước 1: **Hiểu về IQR (Interquartile Range)**

- Hãy tưởng tượng bạn có một danh sách điểm số của các bạn trong lớp được sắp xếp từ thấp đến cao.
- **Q1 (Quartile 1)**: Là điểm số tại vị trí 25% đầu tiên của danh sách (tức là 1/4 số điểm thấp nhất).
- **Q3 (Quartile 3)**: Là điểm số tại vị trí 75% của danh sách (tức là 1/4 số điểm cao nhất).
- **IQR** chính là khoảng cách giữa **Q3** và **Q1**. Nói cách khác:

  \[
  \text{IQR} = Q3 - Q1
  \]

### Bước 2: **Tìm giới hạn dưới và giới hạn trên**

Sau khi có IQR, chúng ta sẽ tính:

- **Giới hạn dưới**: \( \text{Q1} - 1.5 \times \text{IQR} \)
- **Giới hạn trên**: \( \text{Q3} + 1.5 \times \text{IQR} \)

Các giá trị nằm ngoài hai giới hạn này sẽ được coi là **outliers**.

---

### Ví dụ siêu đơn giản

Giả sử bạn có danh sách chiều cao của các bạn như sau:

```
150, 152, 155, 156, 158, 160, 162, 200
```

- **Q1** là 152 (vị trí 25%)
- **Q3** là 160 (vị trí 75%)
- **IQR = 160 - 152 = 8**

Tính giới hạn:

- **Giới hạn dưới**: \( 152 - 1.5 \times 8 = 140 \)
- **Giới hạn trên**: \( 160 + 1.5 \times 8 = 172 \)

Vậy chiều cao **200 cm** là **outlier** vì nó vượt qua giới hạn trên (172).

---

### Bước 3: **Hình ảnh minh họa**

1. **Box Plot**: Giúp nhìn rõ **outliers**. Các chấm nhỏ nằm ngoài "hộp" là outliers.
2. **Biểu đồ phân phối (Histogram)**: Xem chiều cao của các bạn trải đều hay tập trung.
3. **Scatter Plot**: Xem các bạn "lạc loài" (outliers) có xuất hiện không.

---

### Tóm lại:

Phương pháp **IQR** giúp chúng ta dễ dàng phát hiện các giá trị "lạc loài" mà không bị ảnh hưởng bởi các giá trị cực kỳ lớn hoặc cực kỳ nhỏ.